## Parte 1

tenemos una colección de bloques representados como x0,y0,z0\~x1,y1,z1 uno por línea, 2,2,2~2,2,2 significa un único cubo.

el suelo está en z=0 así que lo mas bajo que puede estar un cubo es en z=1. Los cubos están originalmente en el aire y nunca rotan.

En primer lugar hay que calcular en que posición estarán los bloques cuando ya se hayan dejado de mover y luego cuales se pueden desintegrar sin que ningún otro bloque caiga.

Cuantos bloques se pueden desintegrar? (de forma independiente)

In [1]:
import numpy as np

In [6]:
def get_input(filename):
    blocks = []
    with open(filename) as file:
        for i, line in enumerate(file):
            corners = line.strip().split('~')
            c1 = [int(c) for c in corners[0].split(',')]
            c2 = [int(c) for c in corners[1].split(',')]
            blocks.append(Block(i, c1[0], c1[1], c1[2], c2[0], c2[1], c2[2]))
    return blocks

In [7]:
class Block:
    def __init__(self, id, x0, y0, z0, x1, y1, z1):
        self.id = id
        self.x0 = x0
        self.y0 = y0
        self.z0 = z0
        self.x1 = x1
        self.y1 = y1
        self.z1 = z1
        
    def __repr__(self):
        return f'Block {self.id}: ({self.x0}, {self.y0}, {self.z0}), ({self.x1}, {self.y1}, {self.z1})'
    
    def overlap(self, other):
        overlap_x = (other.x0 <= self.x0 and self.x0 <= other.x1) or\
                    (other.x0 <= self.x1 and self.x1 <= other.x1) or\
                    (self.x0 <= other.x0 and other.x0 <= self.x1) or\
                    (self.x0 <= other.x1 and other.x1 <= self.x1)
        overlap_y = (other.y0 <= self.y0 and self.y0 <= other.y1) or\
                    (other.y0 <= self.y1 and self.y1 <= other.y1) or\
                    (self.y0 <= other.y0 and other.y0 <= self.y1) or\
                    (self.y0 <= other.y1 and other.y1 <= self.y1)
        overlap_z = (other.z0 <= self.z0 and self.z0 <= other.z1) or\
                    (other.z0 <= self.z1 and self.z1 <= other.z1) or\
                    (self.z0 <= other.z0 and other.z0 <= self.z1) or\
                    (self.z0 <= other.z1 and other.z1 <= self.z1)
        
        return overlap_x and overlap_y and overlap_z
    
    def move(self, dx=0, dy=0, dz=0):
        self.x0 += dx
        self.y0 += dy
        self.z0 += dz
        self.x1 += dx
        self.y1 += dy
        self.z1 += dz

In [8]:
def caer(blocks):
    blocks.sort(key= lambda x: x.z0)
    
    change = True
    count = 0
    while change:
        count += 1
        change = False
        
        for b in blocks:
            if b.z0 <= 1:
                continue
                
            newb = Block(b.id, b.x0, b.y0, b.z0, b.x1, b.y1, b.z1)
            newb.move(dz=-1)
            
            is_overlap = False
            for o in blocks:
                if newb.id == o.id:
                    continue
                is_overlap = newb.overlap(o)
                if is_overlap:
                    break
            if not is_overlap:
                b.move(dz=-1)
                change = True

In [13]:
def desintegrate(blocks):
    dic_tops = {}
    dic_bots = {}
    
    for b in blocks:
        dic_tops[b.z1] = dic_tops.get(b.z1, []) + [b]
        dic_bots[b.z0] = dic_bots.get(b.z0, []) + [b]
        
    contador = 0
    for b in blocks:
        topz = b.z1
        
        posibles_caidas = dic_bots.get(topz + 1, [])
        
        se_cae_algo = False
        for pc in posibles_caidas:
            posibles_soportes = dic_tops.get(topz, [])
            
            newb = Block(-1, pc.x0, pc.y0, pc.z0 - 1, pc.x1, pc.y1, pc.z1 - 1)
            se_cae = True
            
            for ps in posibles_soportes:
                if ps.id == b.id:
                    continue
                    
                if newb.overlap(ps):
                    se_cae = False
                    break
                    
            if se_cae:
                se_cae_algo = True
                break
                
        if not se_cae_algo:
            contador += 1
    return contador

In [15]:
blocks = get_input('test.txt')
caer(blocks)
desintegrables = desintegrate(blocks)
desintegrables

5

In [16]:
blocks = get_input('input.txt')
caer(blocks)
desintegrables = desintegrate(blocks)
desintegrables

428

## Parte 2

para cada bloque determinar cuantos otros bloques caerían si ese se desintegra, devolver la suma de los bloques que caerían.

In [17]:
def get_tiene_encima(blocks, dic_tops, dic_bots):
    tiene_encima = {}
    
    for b in blocks:
        newb = Block(-1, b.x0, b.y0, b.z0 + 1, b.x1, b.y1, b.z1 + 1)
        tiene_encima[b.id] = set([])
        
        for t in dic_bots.get(b.z1 + 1, []):
            if t.overlap(newb):
                tiene_encima[b.id].add(t)
    return tiene_encima

In [24]:
def get_tiene_debajo(blocks, dic_tops, dic_bots):
    tiene_debajo = {}
    
    for b in blocks:
        newb = Block(-1, b.x0, b.y0, b.z0 - 1, b.x1, b.y1, b.z1 - 1)
        tiene_debajo[b.id] = set([])
        
        for t in dic_tops.get(b.z0 - 1, []):
            if t.overlap(newb):
                tiene_debajo[b.id].add(t)
    return tiene_debajo

In [25]:
def chain_reaction(blocks):
    blocks.sort(key = lambda x: x.id)
    dic_tops = {}    
    dic_bots = {}
    
    for b in blocks:
        dic_tops[b.z1] = dic_tops.get(b.z1, []) + [b]
        dic_bots[b.z0] = dic_bots.get(b.z0, []) + [b]
        
    tiene_debajo = get_tiene_debajo(blocks, dic_tops, dic_bots)
    tiene_encima = get_tiene_encima(blocks, dic_tops, dic_bots)
    
    total_caidos = 0
    
    for b in blocks:
        cola = [b]
        caidos = set([b])
        
        while cola:
            curr = cola.pop()
            
            for be in tiene_encima.get(curr.id, []):
                apoyos = tiene_debajo.get(be.id, [])
                
                if apoyos.issubset(caidos):
                    caidos.add(be)
                    cola.append(be)
                    
        total_caidos += len(caidos) - 1 #el inicial no se cae sino que se desintegra
    return total_caidos

In [26]:
blocks = get_input('test.txt')
caer(blocks)
total_caidos = chain_reaction(blocks)
total_caidos

7

In [27]:
%%time
blocks = get_input('input.txt')
caer(blocks)
total_caidos = chain_reaction(blocks)
total_caidos

CPU times: user 1min 52s, sys: 0 ns, total: 1min 52s
Wall time: 1min 52s


35654